In [10]:
%pip install dotenv pandas langchain openai chromadb torch langchain_huggingface

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached langchain_huggingface-0.1.2-py3-none-any.whl.metadata (1.3 kB)
Using cached langchain_huggingface-0.1.2-py3-none-any.whl (21 kB)
Note: you may need to restart the kernel to use updated packages.


## RAG로 사용할 데이터 확인

### 1. 도서 정보 데이터

데이터는 공공API에서 제공하는 [국립중앙도서관 사서추천도서목록](https://www.nl.go.kr/NL/contents/N31101030900.do)을 사용한다.

xml 데이터를 json 데이터로 변환하였다.

In [2]:
import json

# JSON 데이터 로드
with open('./data/library_books.json', 'r', encoding='utf-8') as f:
    books = json.load(f)

print("\n" + "="*50)
print("📚 데이터 구조")
print("="*50)
print("\n📍 데이터 키 목록:")
print(", ".join(list(books[0].keys())))

print("\n📍 첫 번째 도서 상세 정보:")
for key, value in books[0].items():
    print(f"\n{key}:")
    # 긴 텍스트는 줄바꿈하여 보기 좋게 출력
    if len(str(value)) > 100:
        # 100자마다 줄바꿈
        formatted_value = '\n'.join([str(value)[i:i+100] for i in range(0, len(str(value)), 100)])
        print(f"    {formatted_value}")
    else:
        print(f"    {value}")

print("\n" + "="*50)
print(f"📚 전체 도서 수: {len(books)}권")
print("="*50 + "\n")


📚 데이터 구조

📍 데이터 키 목록:
category_code, category_name, title, author, publisher, isbn, contents, table_of_contents, publish_year, recommend_year, recommend_month

📍 첫 번째 도서 상세 정보:

category_code:
    6

category_name:
    인문과학

title:
    제대로 연습하는 법 : 어학부터 스포츠까지, 인지심리학이 제시하는 배움의 기술

author:
    아투로 E. 허낸데즈 지음 ;방진이 옮김

publisher:
    북트리거 지학사

isbn:
    9791193378335

contents:
    한때 유행했던 일만 시간의 법칙을 기억하는가? 어떤 분야에서 전문가가 되려면 최소한 일만 시간의 훈련이 필요하다는 개념이다. 하지만 단순히 연습의 양이 많다고 해서 모두가 전문가가
 되는 것은 아니다. 이 책은 심리학자이자 다중언어 구사자, 테니스 선수로도 활약했던 저자가 학습과 훈련, 그리고 기량 향상의 상관관계를 연구한 결과를 담고 있다. 장 피아제, 노
엄 촘스키, 그리고 일만 시간의 법칙을 제창한 심리학자 안데르스 에릭손 등 대가들의 이론 및 최신 연구 결과를 바탕으로, 뇌과학과 인지심리학적 관점에서 '제대로 연습하는 법'을 탐
구한다. 아마추어 스포츠 선수, 유명 체스 선수, 다중언어 구사자, 피아노 연주자 등 다양한 사례 분석을 통해 연습의 물리적 양보다 중요한 것은 질적인 측면임을 강조한다. 적절한 
휴식 속에서 배운 것을 재조합하고, 몰입 상태에서 연습할 때 비로소 '최고'라는 목표에 도달할 수 있는 것이다. 벌써 2025년이 100일 넘게 흘렀다. 완연한 봄을 맞이하여 새로
운 기술을 배우고 싶거나, 그동안 노력에 비해 실력이 늘지 않는다고 느껴왔다면, 이 책을 길잡이 삼아 ‘제대로 연습하는 법’을 배워보면 어떨까?

table_of_contents:
    서론 : 작은 

category 정보를 확인한다.

In [3]:
from collections import defaultdict
import pandas as pd
from IPython.display import display

# 카테고리 코드와 이름을 매핑하여 저장
category_map = defaultdict(int)
for book in books:
    category_key = (book['category_code'], book['category_name'])
    category_map[category_key] += 1

# 데이터프레임으로 변환
df_categories = pd.DataFrame([
    {
        '분류 코드': code,
        '분류명': name,
        '도서 수': count
    }
    for (code, name), count in category_map.items()
])

# 도서 수를 기준으로 내림차순 정렬
df_categories = df_categories.sort_values('도서 수', ascending=False)

# 스타일 적용
styled_df = df_categories.style\
    .set_properties(**{'text-align': 'center'})\
    .set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'center'), ('background-color', '#f0f0f0')]},
        {'selector': 'td', 'props': [('text-align', 'center')]},
    ])\
    .format({'도서 수': '{:,d}'})  # 천 단위 구분자 추가

# 테이블 출력
print(f"\n📚 총 분류 수: {len(df_categories)}개\n")
display(styled_df)


📚 총 분류 수: 6개



,분류 코드,분류명,도서 수
0,6,인문과학,342
1,5,사회과학,338
3,4,자연과학,319
2,11,어문학,296
4,425,,92
5,8,,1


chuck size를 설정하기 위해 텍스트 길이를 파악했다.

임베딩할 데이터는 category_name, title, contents, table_of_contents이다.

In [4]:
import pandas as pd

# 각 도서별 텍스트 길이 계산
text_lengths = []
for book in books:
    text = f"카테고리: {book['category_name']}\n제목: {book['title']}\n내용: {book['contents']}\n목차: {book['table_of_contents']}"
    length_info = {
        '카테고리': len(book['category_name']),
        '제목': len(book['title']),
        '내용': len(book['contents']),
        '목차': len(book['table_of_contents']),
        '전체': len(text)
    }
    text_lengths.append(length_info)

# DataFrame으로 변환
df_lengths = pd.DataFrame(text_lengths)

# 기본 통계 계산
stats = df_lengths.agg(['min', 'max', 'mean', 'median', 'std']).round(2)

# 결과 출력
print("\n=== 각 필드별 텍스트 길이 통계 ===")
display(stats)

# 백분위수 계산
percentiles = df_lengths['전체'].quantile([0.25, 0.5, 0.75, 0.9])
print("\n=== 전체 텍스트 길이 백분위수 ===")
for p, v in percentiles.items():
    print(f"{int(p*100)}번째 백분위: {int(v)}자")


=== 각 필드별 텍스트 길이 통계 ===


,카테고리,제목,내용,목차,전체
min,0.00,1.00,135.00,0.00,323.00
max,4.00,75.00,861.00,6677.00,7296.00
mean,3.52,14.16,495.10,884.01,1417.79
median,4.00,11.00,490.00,734.50,1264.50
std,1.03,9.65,95.41,740.71,746.49



=== 전체 텍스트 길이 백분위수 ===
25번째 백분위: 881자
50번째 백분위: 1264자
75번째 백분위: 1758자
90번째 백분위: 2343자


### 2. 사용자 정보

사용자 정보는 사용자가 읽은 책 정보와 사용자의 책 취향 정보를 사용한다.

#### 1. 사용자가 읽은 책 정보

In [5]:
with open('./data/test-case/user_reading_history.json', 'r', encoding='utf-8') as f:
    reading_history = json.load(f)

for i, book in enumerate(reading_history['read_books'], 1):
    print(f"\n📚 도서 {i}:")
    for key, value in book.items():
        print(f"  {key}: {value}")


📚 도서 1:
  title: 완벽이라는 중독 : 불안한 완벽주의자를 위한 심리학
  author: 토머스 커런 지음 ;김문주 옮김
  rating: 4.5
  review: 심리학적 관점에서 자아를 이해하는데 도움이 됨
  read_date: 2024-01-05
  genre: ['심리학', '자기계발']

📚 도서 2:
  title: 침묵을 배우는 시간 : 말이 넘쳐나는 세상 속, 더욱 빛을 발하는 침묵의 품격
  author: 코르넬리아 토프 지음 ;장혜경 옮김
  rating: 4.0
  review: 따뜻한 위로가 되는 에세이
  read_date: 2024-01-15
  genre: ['에세이', '자기계발']

📚 도서 3:
  title: 나답게 산다는 것 : 나를 찾고자 하는 이들의 철학수업
  author: 박은미 지음
  rating: 4.5
  review: 쉽게 읽을 수 있는 철학 에세이
  read_date: 2024-01-25
  genre: ['철학', '에세이']

📚 도서 4:
  title: 출근길 심리학 : 단단하고 유연한 멘탈을 위한 33가지 마음의 법칙
  author: 반유화 지음
  rating: 3.5
  review: 일상적인 심리학 이야기
  read_date: 2024-02-05
  genre: ['심리학', '자기계발']

📚 도서 5:
  title: 내가 누구인지 아는 것이 왜 중요한가
  author: 페터 베르 지음 ;장혜경 옮김
  rating: 4.0
  review: 자아 성찰에 도움이 되는 책
  read_date: 2024-02-15
  genre: ['심리학', '에세이']

📚 도서 6:
  title: 자화상 내 마음을 그리다
  author: 김선현 지음
  rating: 3.5
  review: 마음을 따뜻하게 해주는 이야기
  read_date: 2024-02-25
  genre: ['에세이']

📚 도서 7:
  title: 각본 없음 : 삶의 다음 페이지로 넘어가기 위해 쓴 

#### 2. 사용자의 책 취향 정보

In [6]:
with open('./data/test-case/user_reading_preferences.json', 'r', encoding='utf-8') as f:
    reading_preferences = json.load(f)

preferences = reading_preferences['preferences']

print("\n📚 선호 장르:")
for genre in preferences['genres']:
    print(f"  - {genre['name']} (가중치: {genre['weight']})")

print("\n📖 독서 스타일:")
for key, value in preferences['reading_style'].items():
    print(f"  - {key}: {value}")

print("\n🔑 관심 키워드:")
print(f"  {', '.join(preferences['keywords'])}")

print("\n⛔ 기피 주제:")
print(f"  {', '.join(preferences['avoid_topics'])}")


📚 선호 장르:
  - 심리학 (가중치: 0.8)
  - 자기계발 (가중치: 0.7)
  - 에세이 (가중치: 0.6)

📖 독서 스타일:
  - preferred_length: 중간
  - complexity_level: 쉬움
  - tone: 따뜻함

🔑 관심 키워드:
  불안, 힐링, 위로, 심리, 일상

⛔ 기피 주제:
  우울, 공포, 긴장감


### 벡터 DB 생성

In [7]:
import torch

# GPU 사용 가능 여부 확인 및 디바이스 설정
if torch.cuda.is_available():
    device = 'cuda'
    device_map = {'': 0}
elif torch.backends.mps.is_available():
    device = 'mps'
    device_map = {'': device}
else:
    device = 'cpu'
    device_map = {'': device}

print(f"사용 디바이스: {device}")

사용 디바이스: mps


In [11]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.vectorstores import Chroma
import json

# 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': device}
)

#--------------------- 1. 도서 정보 벡터 DB ---------------------
with open('./data/library_books.json', 'r', encoding='utf-8') as f:
    books = json.load(f)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=300,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)

book_documents = []
for book in books:
    text = f"카테고리: {book['category_name']}\n제목: {book['title']}\n내용: {book['contents']}\n목차: {book['table_of_contents']}"
    
    metadata = {
        'title': book['title'],
        'category': book['category_name'],
        'author': book['author'],
        'isbn': book['isbn'],
        'publish_year': book['publish_year'],
    }
    
    chunks = text_splitter.create_documents(
        texts=[text],
        metadatas=[metadata]
    )
    book_documents.extend(chunks)

# 도서 정보 벡터 DB 생성
books_vectorstore = Chroma.from_documents(
    documents=book_documents,
    embedding=embeddings,
    collection_name="books_collection"
)

print(f"도서 정보 벡터 DB 생성 완료: {len(book_documents)}개 문서")

#--------------------- 2. 독서 이력 벡터 DB ---------------------
with open('./data/test-case/user_reading_history.json', 'r', encoding='utf-8') as f:
    reading_history = json.load(f)

history_documents = []
for book in reading_history['read_books']:
    text = f"제목: {book['title']}\n저자: {book['author']}\n평점: {book['rating']}\n리뷰: {book['review']}\n장르: {', '.join(book['genre'])}\n읽은 날짜: {book['read_date']}"
    
    metadata = {
        'title': book['title'],
        'author': book['author'],
        'rating': book['rating'],
        'genre_str': ', '.join(book['genre']),
        'read_date': book['read_date']
    }
    
    doc = Document(page_content=text, metadata=metadata)
    history_documents.append(doc)

# 독서 이력 벡터 DB 생성
history_vectorstore = Chroma.from_documents(
    documents=history_documents,
    embedding=embeddings,
    collection_name="reading_history_collection"
)

print(f"독서 이력 벡터 DB 생성 완료: {len(history_documents)}개 문서")

#--------------------- 3. 독서 취향 벡터 DB ---------------------
with open('./data/test-case/user_reading_preferences.json', 'r', encoding='utf-8') as f:
    reading_preferences = json.load(f)

preferences_text = f"선호 장르: {', '.join([f'{g['name']}({g['weight']})' for g in reading_preferences['preferences']['genres']])}\n"
preferences_text += f"독서 스타일: 길이({reading_preferences['preferences']['reading_style']['preferred_length']}), "
preferences_text += f"복잡도({reading_preferences['preferences']['reading_style']['complexity_level']}), "
preferences_text += f"톤({reading_preferences['preferences']['reading_style']['tone']})\n"
preferences_text += f"관심 키워드: {', '.join(reading_preferences['preferences']['keywords'])}\n"
preferences_text += f"기피 주제: {', '.join(reading_preferences['preferences']['avoid_topics'])}"

preferences_metadata = {
    'genres_str': ', '.join([g['name'] for g in reading_preferences['preferences']['genres']]),
    'keywords_str': ', '.join(reading_preferences['preferences']['keywords']),
    'avoid_topics_str': ', '.join(reading_preferences['preferences']['avoid_topics'])
}

preferences_doc = Document(page_content=preferences_text, metadata=preferences_metadata)

# 독서 취향 벡터 DB 생성 (단일 문서)
preferences_vectorstore = Chroma.from_documents(
    documents=[preferences_doc],
    embedding=embeddings,
    collection_name="reading_preferences_collection"
)

print("독서 취향 벡터 DB 생성 완료: 1개 문서")


도서 정보 벡터 DB 생성 완료: 2201개 문서
독서 이력 벡터 DB 생성 완료: 10개 문서
독서 취향 벡터 DB 생성 완료: 1개 문서


### 추천 모델 구현

공통 프롬프트를 작성한다.

In [12]:
from langchain.prompts import ChatPromptTemplate

# 공통 기본 프롬프트 템플릿
BASE_RECOMMENDATION_TEMPLATE = """당신은 사용자의 독서 취향과 감정 상태에 맞춰 도서를 추천하는 전문가입니다.
다음 정보를 분석하여 사용자에게 가장 적합한 {num_recommendations}권의 책을 추천해주세요.

## 사용자 정보
- 현재 감정 상태: {user_emotion}
- 원하는 감정적 효과: {desired_emotional_effect}
- 직업: {occupation}
- 독서 상황: {reading_context}
- 선호하는 집중도: {focus_level}

## 사용자 취향 정보
{preferences}

## 사용자가 읽은 관련 책들
{reading_history}

## 후보 도서 목록
{candidate_books}

다음과 같은 척도로 각 후보 도서를 평가하세요:
1. 사용자의 현재 감정 상태와 원하는 감정적 효과와의 적합성
2. 사용자의 취향과의 일치도 (장르, 스타일, 선호 키워드)
3. 사용자의 기피 주제와 겹치지 않는지 여부
4. 사용자의 독서 이력과의 연관성
5. 독서 상황과 선호하는 집중도에 맞는지 여부
6. 사용자의 직업과 관련된 통찰이나 도움이 될 수 있는지 여부

최종 추천은 다음 형식으로 제시해주세요:
1. [첫 번째 추천 도서 제목] - 저자
   - 추천 이유: (사용자의 현재 감정과 원하는 효과를 고려한 구체적인 이유)
   - 이 책이 도움이 될 수 있는 이유: (감정적 효과나 얻을 수 있는 통찰 등)

2. [두 번째 추천 도서 제목] - 저자
   - 추천 이유: (사용자의 현재 감정과 원하는 효과를 고려한 구체적인 이유)
   - 이 책이 도움이 될 수 있는 이유: (감정적 효과나 얻을 수 있는 통찰 등)

3. [세 번째 추천 도서 제목] - 저자
   - 추천 이유: (사용자의 현재 감정과 원하는 효과를 고려한 구체적인 이유)
   - 이 책이 도움이 될 수 있는 이유: (감정적 효과나 얻을 수 있는 통찰 등)
"""

# 공통 프롬프트 템플릿 생성
base_prompt_template = ChatPromptTemplate.from_template(BASE_RECOMMENDATION_TEMPLATE)

# 포맷팅 함수는 동일하게 유지
def format_reading_history(reading_history_data):
    if not reading_history_data:
        return "관련 독서 이력이 없습니다."
    
    # 다단계 RAG 결과 포맷팅
    if isinstance(reading_history_data, list) and all(isinstance(item, dict) and 'metadata' in item for item in reading_history_data):
        return "\n\n".join([
            f"- 도서: {item['metadata'].get('title', '제목 없음')}\n"
            f"  저자: {item['metadata'].get('author', '저자 미상')}\n"
            f"  장르: {item['metadata'].get('genre_str', '장르 없음')}\n"
            f"  평점: {item['metadata'].get('rating', 0)}\n"
            f"  내용: {item['content'][:200]}..."
            for item in reading_history_data
        ])
    
    # 직접 벡터 검색 결과 포맷팅
    elif isinstance(reading_history_data, list) and all(isinstance(item, dict) and 'title' in item for item in reading_history_data):
        return "\n".join([
            f"- '{item['title']}' (저자: {item['author']}, 장르: {item['genre']}, 평점: {item['rating']})\n  리뷰: {item['review']}"
            for item in reading_history_data
        ])
    
    # 다른 형식의 데이터
    return str(reading_history_data)

def format_book_candidates(candidates_data):
    if not candidates_data:
        return "추천할 만한 후보 도서가 없습니다."
    
    # 다단계 RAG 결과 포맷팅
    if isinstance(candidates_data, list) and all(isinstance(item, dict) and 'metadata' in item for item in candidates_data):
        return "\n\n".join([
            f"{i+1}. 제목: {item['metadata'].get('title', '제목 없음')}\n"
            f"   저자: {item['metadata'].get('author', '저자 미상')}\n"
            f"   카테고리: {item['metadata'].get('category', '카테고리 없음')}\n"
            f"   내용: {item['content'][:300]}..."
            for i, item in enumerate(candidates_data)
        ])
    
    # 직접 벡터 검색 결과 포맷팅
    elif isinstance(candidates_data, list) and all(isinstance(item, dict) and 'index' in item for item in candidates_data):
        return "\n".join([
            f"{book['index']}. '{book['title']}' (저자: {book['author']}, 카테고리: {book['category']})\n   내용 요약: {book['content'][:200]}..."
            for book in candidates_data
        ])
    
    # 다른 형식의 데이터
    return str(candidates_data)


llm 모델을 정의한다.

In [13]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    top_p=0.9,
    max_tokens=512,
    api_key=api_key
)

chain = base_prompt_template | llm

#### 1. RAG에 기반한 추천 로직

In [15]:
def multi_stage_rag_search(user_query, num_candidates, num_history_results):
    results = {}
    
    # 1단계: 항상 사용자 취향 정보 검색 (가장 중요)
    preference_results = preferences_vectorstore.similarity_search_with_score(user_query, k=1)
    if preference_results:
        results['user_preferences'] = preference_results[0]
    
    # 2단계: 사용자의 독서 이력에서 관련 도서 검색
    history_results = history_vectorstore.similarity_search_with_score(user_query, k=num_history_results)
    if history_results:
        results['reading_history'] = history_results
    
    # 3단계: 독서 이력과 취향 정보를 고려하여 강화된 쿼리 생성
    enhanced_query = user_query
    
    # 취향 정보에서 관련 키워드 추출
    if 'user_preferences' in results:
        pref_doc = results['user_preferences'][0]
        keywords = pref_doc.metadata.get('keywords_str', '')
        if keywords:
            enhanced_query += f" 키워드: {keywords}"
    
    # 독서 이력에서 높은 평점의 장르 추출
    if 'reading_history' in results:
        liked_genres = []
        for doc, _ in results['reading_history']:
            if doc.metadata.get('rating', 0) >= 4.0:
                genres = doc.metadata.get('genre_str', '').split(', ')
                liked_genres.extend(genres)
        
        if liked_genres:
            unique_genres = list(set(liked_genres))
            enhanced_query += f" 선호 장르: {', '.join(unique_genres[:3])}"
    
    print(f"강화된 쿼리: {enhanced_query}")
    
    # 4단계: 강화된 쿼리로 도서 검색
    book_results = books_vectorstore.similarity_search_with_score(enhanced_query, k=num_candidates)
    results['recommended_books'] = book_results
    
    # 결과 포맷팅 및 반환
    formatted_results = []
    
    # 사용자 취향 정보
    if 'user_preferences' in results:
        pref_doc, pref_score = results['user_preferences']
        formatted_results.append({
            'type': 'user_preferences',
            'content': pref_doc.page_content,
            'metadata': pref_doc.metadata,
            'score': pref_score
        })
    
    # 관련 독서 이력
    if 'reading_history' in results:
        for i, (hist_doc, hist_score) in enumerate(results['reading_history']):
            if i < 2:  # 최대 2개만 포함
                formatted_results.append({
                    'type': 'reading_history',
                    'content': hist_doc.page_content,
                    'metadata': hist_doc.metadata,
                    'score': hist_score
                })
    
    # 추천 도서
    recommended_count = 0
    for book_doc, book_score in results['recommended_books']:
        formatted_results.append({
            'type': 'recommended_book',
            'content': book_doc.page_content,
            'metadata': book_doc.metadata,
            'score': book_score
        })
        
        recommended_count += 1
        if recommended_count >= num_candidates - 2:  # 취향과 이력을 포함하고 남은 슬롯 채우기
            break
    
    return formatted_results

In [16]:
def multi_stage_recommendation(user_emotion, desired_emotional_effect, occupation, reading_context, focus_level, num_candidates=8, num_history_results=3, num_recommendations=3):
    """다단계 RAG 검색 결과를 바탕으로 LLM에 도서 추천을 요청하는 함수"""
    # 감정과 원하는 효과를 기반으로 검색 쿼리 구성
    search_query = f"{user_emotion}을 느끼는 사람이 {desired_emotional_effect}할 수 있는 책"
    
    # 다단계 RAG 검색 수행
    rag_results = multi_stage_rag_search(search_query, num_candidates, num_history_results)
    
    # 결과를 타입별로 분류
    user_preferences = None
    reading_history = []
    book_candidates = []
    
    for item in rag_results:
        if item['type'] == 'user_preferences':
            user_preferences = item
        elif item['type'] == 'reading_history':
            reading_history.append(item)
        elif item['type'] == 'recommended_book':
            book_candidates.append(item)
    
    # 취향 정보 포맷팅
    preferences_formatted = user_preferences['content'] if user_preferences else "취향 정보가 없습니다."
    
    # 독서 이력 및 후보 도서 포맷팅 (공통 함수 사용)
    history_formatted = format_reading_history(reading_history)
    candidates_formatted = format_book_candidates(book_candidates)
    
    result = chain.invoke({
        "user_emotion": user_emotion,
        "desired_emotional_effect": desired_emotional_effect,
        "occupation": occupation,
        "reading_context": reading_context,
        "focus_level": focus_level,
        "preferences": preferences_formatted,
        "reading_history": history_formatted,
        "candidate_books": candidates_formatted,
        "num_recommendations": num_recommendations
    })
    
    return result.content

#### 2. Prompt 기반의 추천 로직

In [17]:
# 취향 정보를 문자열로 포맷팅 (프롬프트에 직접 사용)
preferences_formatted = f"""
## 사용자 독서 취향 정보
- 선호 장르: {', '.join([f"{g['name']}(가중치:{g['weight']})" for g in reading_preferences['preferences']['genres']])}
- 선호 독서 스타일: 
  * 선호 길이: {reading_preferences['preferences']['reading_style']['preferred_length']}
  * 복잡도: {reading_preferences['preferences']['reading_style']['complexity_level']}
  * 선호 톤: {reading_preferences['preferences']['reading_style']['tone']}
- 관심 키워드: {', '.join(reading_preferences['preferences']['keywords'])}
- 기피 주제: {', '.join(reading_preferences['preferences']['avoid_topics'])}
"""

print("사용자 취향 정보 추출 완료")

사용자 취향 정보 추출 완료


In [18]:
def direct_vector_recommendation(user_emotion, desired_emotional_effect, occupation, reading_context, focus_level, num_candidates=8, num_history_results=3, num_recommendations=3):
    """벡터 DB 직접 검색 결과를 바탕으로 LLM에 도서 추천을 요청하는 함수"""
    # 감정과 원하는 효과를 기반으로 검색 쿼리 구성
    search_query = f"{user_emotion}을 느끼는 사람이 {desired_emotional_effect}할 수 있는 책"
    
    # 1. 쿼리에 관련된 도서 검색
    book_results = books_vectorstore.similarity_search_with_score(search_query, k=num_candidates)
    
    # 2. 사용자의 독서 이력 중 관련 있는 책 검색
    history_results = history_vectorstore.similarity_search_with_score(search_query, k=num_history_results)
    
    # 3. 검색 결과 포맷팅
    candidate_books = []
    for i, (doc, score) in enumerate(book_results):
        book_info = {
            'index': i+1,
            'title': doc.metadata.get('title', '제목 없음'),
            'author': doc.metadata.get('author', '저자 미상'),
            'category': doc.metadata.get('category', '분류 없음'),
            'content': doc.page_content.strip()[:500] + "..." if len(doc.page_content) > 500 else doc.page_content.strip(),
            'similarity_score': score
        }
        candidate_books.append(book_info)
    
    reading_history_formatted = []
    for doc, score in history_results:
        history_info = {
            'title': doc.metadata.get('title', '제목 없음'),
            'author': doc.metadata.get('author', '저자 미상'),
            'genre': doc.metadata.get('genre_str', '장르 없음'),
            'rating': doc.metadata.get('rating', 0),
            'review': doc.page_content.split('리뷰: ')[1].split('\n')[0] if '리뷰: ' in doc.page_content else '리뷰 없음'
        }
        reading_history_formatted.append(history_info)
    
    # 독서 이력 및 후보 도서 포맷팅 (공통 함수 사용)
    history_formatted = format_reading_history(reading_history_formatted)
    candidates_formatted = format_book_candidates(candidate_books)
    
    result = chain.invoke({
        "user_emotion": user_emotion,
        "desired_emotional_effect": desired_emotional_effect,
        "occupation": occupation,
        "reading_context": reading_context,
        "focus_level": focus_level,
        "preferences": preferences_formatted,  # 사용자 취향 정보 (전역 변수)
        "reading_history": history_formatted,
        "candidate_books": candidates_formatted,
        "num_recommendations": num_recommendations
    })
    
    return result.content

In [19]:
def get_book_recommendations(user_emotion, desired_emotional_effect, occupation, reading_context, focus_level, method="both"):
    print(f"현재 감정 상태: {user_emotion}")
    print(f"원하는 감정적 효과: {desired_emotional_effect}")
    print(f"직업: {occupation}")
    print(f"독서 상황: {reading_context}")
    print(f"선호하는 집중도: {focus_level}")
    
    results = {}

    num_candidates = 6
    num_history_results = 3
    num_recommendations = 3
    
    if method in ["multi_stage", "both"]:
        print("\n다단계 RAG 기반 추천 도서를 찾는 중...\n")
        multi_stage_result = multi_stage_recommendation(
            user_emotion=user_emotion,
            desired_emotional_effect=desired_emotional_effect,
            occupation=occupation,
            reading_context=reading_context,
            focus_level=focus_level,
            num_candidates=num_candidates,
            num_history_results=num_history_results,
            num_recommendations=num_recommendations
        )
        results["multi_stage"] = multi_stage_result
    
    if method in ["direct", "both"]:
        print("\n직접 벡터 검색 기반 추천 도서를 찾는 중...\n")
        direct_result = direct_vector_recommendation(
            user_emotion=user_emotion,
            desired_emotional_effect=desired_emotional_effect,
            occupation=occupation,
            reading_context=reading_context,
            focus_level=focus_level,
            num_candidates=num_candidates,
            num_history_results=num_history_results,
            num_recommendations=num_recommendations
        )
        results["direct"] = direct_result
    
    return results

In [20]:
# 테스트 시나리오 (README.md의 TestCase1에 맞게 수정)
test_results = get_book_recommendations(
    user_emotion="불안함",                      # 질문 1 응답
    desired_emotional_effect="위로와 안정감",     # 질문 2 응답
    occupation="직장인",                       # 질문 3 응답
    reading_context="잠들기 전 독서",            # 질문 4 응답
    focus_level="가볍게 읽을 수 있는 책",         # 질문 5 응답
    method="both"  # 두 방식 모두 사용
)

print("\n=== 다단계 RAG 기반 도서 추천 결과 ===\n")
print(test_results.get("multi_stage", "결과 없음"))

print("\n=== 직접 벡터 검색 기반 도서 추천 결과 ===\n")
print(test_results.get("direct", "결과 없음"))

현재 감정 상태: 불안함
원하는 감정적 효과: 위로와 안정감
직업: 직장인
독서 상황: 잠들기 전 독서
선호하는 집중도: 가볍게 읽을 수 있는 책

다단계 RAG 기반 추천 도서를 찾는 중...

강화된 쿼리: 불안함을 느끼는 사람이 위로와 안정감할 수 있는 책 키워드: 불안, 힐링, 위로, 심리, 일상 선호 장르: 에세이, 여행

직접 벡터 검색 기반 추천 도서를 찾는 중...


=== 다단계 RAG 기반 도서 추천 결과 ===

1. **[불안해 보여서 불안한 당신에게] - 한창욱**
   - 추천 이유: 이 책은 불안한 청춘들을 위로하고, 그들의 감정을 이해해 주는 에피소드들로 구성되어 있습니다. 사용자가 현재 느끼고 있는 불안감에 공감하며, 이를 덜어줄 수 있는 따뜻한 메시지를 담고 있습니다. 
   - 이 책이 도움이 될 수 있는 이유: 다양한 사례를 통해 불안의 원인과 그에 대한 대처 방안을 제시하므로, 사용자가 느끼는 감정에 대한 이해와 위로를 제공할 수 있습니다. 또한, 일상에서의 불안감을 해소하는 데 필요한 통찰을 얻을 수 있습니다.

2. **[잠깐 머리 좀 식히고 오겠습니다] - 윤대현**
   - 추천 이유: 이 책은 스트레스 관리와 마음의 안정을 위한 심리 처방전을 제공하며, 직장인으로서의 일상에서 느끼는 불안감을 이해하고 극복하는 데 도움을 줄 수 있습니다. 
   - 이 책이 도움이 될 수 있는 이유: 저자가 제시하는 다양한 팁과 조언은 직장 생활에서의 스트레스를 줄이고 자존감을 높이는 데 유용하며, 가벼운 문체로 쉽게 읽을 수 있어 잠들기 전 독서에 적합합니다.

3. **[자화상 내 마음을 그리다] - 김선현**
   - 추천 이유: 사용자가 이미 읽고 긍정적인 평가를 한 책으로, 따뜻한 이야기로 마음을 위로해 줄 수 있는 내용을 담고 있습니다. 이 책은 감정의 깊이를 이해하고 일상에서 느끼는 불안을 해소하는 데 도움을 줄 것입니다.
   - 이 책이 도움이 될 수 있는 이유: 이미 긍정적인 경험이 있는 책이므로, 비슷한 느낌의